In [105]:
import torch
import numpy as np
import pandas as pd
import torch.optim as optim
from torch import nn
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split
from sklearn.metrics import ndcg_score
import networkx as ne
from node2vec import Node2Vec
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [106]:
# 读loaded_data取保存的 CSV 文件
loaded_data = pd.read_csv('book_score.csv')

# 显示加载的数据
#print(loaded_data)
loaded_data=loaded_data[loaded_data['Rate'] != 0]
print(loaded_data)

           User     Book  Rate                       Time          Tag
45      1398478  2348372     4  2009-11-10T18:42:00+08:00          NaN
164     1779492  1851385     3  2011-03-13T12:37:12+08:00  奥尔罕·帕慕克,土耳其
165     1779492  3266345     3  2010-10-20T19:31:20+08:00      葛瑞格·摩顿森
166     1779492  1001885     3  2010-10-20T19:29:16+08:00     林达,法国,旅行
168     1779492  1424741     3  2010-10-04T01:24:33+08:00      卡森·麦卡勒斯
...         ...      ...   ...                        ...          ...
637249  4507957  1125186     4  2009-07-04T08:02:13+08:00   张爱玲,半生缘,爱情
637250  4507957  1002299     5  2009-07-04T08:01:28+08:00   金庸,武侠,笑傲江湖
637251  4507957  1001136     4  2009-07-04T07:55:17+08:00      彼得・潘,童话
637252  4507957  1021615     5  2009-07-04T07:53:54+08:00    小王子,童话,经典
637253  4507957  1962929     5  2009-06-29T22:13:37+08:00           爱情

[403807 rows x 5 columns]


In [107]:
#TODO 处理加载的数据，得到item、user和star信息。
#合并相同的userids和itemsids
user_ids=loaded_data["User"].unique()
item_ids=loaded_data["Book"].unique()
#创建user to row的字典
user_to_row ={user_id : idx for idx, user_id in enumerate(user_ids)}
item_to_row ={item_id : idx for idx, item_id in enumerate(item_ids)}

In [108]:
# 初始化一个空字典来存储社交关系
contact = {}

# 打开文件并读取内容
with open('Contacts.txt', 'r', encoding='utf-8') as f:
    for line in f:
        # 分割每一行的内容
        user, friends = line.strip().split(':')
        # 将朋友列表转换为整数列表
        if int(user) in user_to_row:
            friends_list = [user_to_row[int(friend)] for friend in friends.split(',') if int(friend) in user_to_row]
            # 将朋友列表添加到字典中
            contact[user_to_row[int(user)]] = friends_list

contact_sorted = {k: v for k, v in sorted(contact.items())}
# 打印字典的内容
print(contact_sorted)

{0: [1504, 3061, 3200, 3554, 3238, 1567, 271], 1: [3766, 3840, 2655, 1655, 1894, 91, 2783, 1420, 2521, 717, 2994], 2: [185, 1194, 2521, 1106, 1597, 4193, 4010, 3951, 3114, 1704, 1027, 4006, 3836, 2570, 1665, 3322, 3657, 32, 1255, 1261, 4171, 3029, 593, 392, 2283, 4102], 3: [1951, 38, 1689, 3893, 3252, 108, 3466, 148, 998, 3694, 1799, 3054, 1928, 265, 3458, 2188, 3780, 3759, 2801, 3868, 3522, 3024, 474, 1261, 1973, 3600, 1539, 114, 3042, 1325, 3968, 1221, 1803, 1992, 1504, 2957, 1369, 1123, 2594, 1027, 3159, 2491, 717, 1680, 1514, 1073, 22], 4: [2324, 998, 3845, 3429, 3042, 2477, 1209, 1261, 539, 2002, 537, 742, 4211, 2828, 3729, 1108, 2040, 2957], 5: [3452, 717, 1099, 29, 1126, 3481, 3060, 3650, 288, 139, 4278], 6: [555, 2601, 1567, 3677, 314, 2679, 3803], 7: [2796, 3422, 2115, 3650, 2655, 2325, 3836, 2720, 3554, 3494, 2802, 265, 2324, 2124, 958, 2440, 1690, 105, 1455, 4305, 1381, 20, 3431, 1565, 1633, 3913, 2431, 212, 2981, 4052, 134, 1221, 1209, 3669, 4278, 446, 1375, 1164, 1309, 261

In [109]:

import networkx as nx

G = nx.Graph()
for key, value in contact_sorted.items():
    for v in value:
        G.add_edge(key, v)
node2vec_model = Node2Vec(G, dimensions=200, walk_length=30, num_walks=200, workers=4,temp_folder="D:/temp")
model = node2vec_model.fit(window=10, min_count=1, batch_words=4)
social_embedding_temp = {node : model.wv[node] for node in G.nodes() if node in model.wv}

Computing transition probabilities:   0%|          | 0/4309 [00:00<?, ?it/s]

Generating walks (CPU: 2):   0%|          | 0/50 [00:00<?, ?it/s]


Generating walks (CPU: 2):   6%|▌         | 3/50 [00:09<02:38,  3.37s/it]


Generating walks (CPU: 2):   8%|▊         | 4/50 [00:14<03:03,  4.00s/it]

Generating walks (CPU: 2):  12%|█▏        | 6/50 [00:26<03:47,  5.16s/it]


Generating walks (CPU: 2):  14%|█▍        | 7/50 [00:32<03:41,  5.16s/it]


Generating walks (CPU: 2):  18%|█▊        | 9/50 [00:43<03:40,  5.38s/it]


Generating walks (CPU: 2):  22%|██▏       | 11/50 [00:52<03:21,  5.16s/it]


Generating walks (CPU: 2):  24%|██▍       | 12/50 [00:58<03:15,  5.14s/it]


Generating walks (CPU: 2):  26%|██▌       | 13/50 [01:04<03:20,  5.42s/it]

Generating walks (CPU: 2):  30%|███       | 15/50 [01:15<03:09,  5.42s/it]


Generating walks (CPU: 2):  34%|███▍      | 17/50 [01:25<02:52,  5.22s/it]

Generating walks (CPU: 2):  38%|███▊      | 19/50 [01:34<02:36,  5.04s/it]


Generating walks (CPU: 2):  40%|████      | 20/50 [01:40<02:31,  5.05s/it]


Generating walks

In [110]:
social_embedding_dict = {}
for key, value in social_embedding_temp.items():
    social_embedding_dict[key] = value.tolist()


In [111]:
import pickle
with open('social_embedding_dict.pkl', 'wb') as f:
    pickle.dump(social_embedding_dict, f)

In [112]:
with open('social_embedding_dict.pkl', 'rb') as f:
    social_embedding_dict = pickle.load(f)

In [113]:

class RatingDataset(Dataset):
    def __init__(self,data,user_to_row,item_to_row,social_embedding_dict):
        self.data=data
        #self.user_item_matrix=user_item_matrix
        self.user_to_row=user_to_row
        self.item_to_row=item_to_row
        self.social_embedding_dict = social_embedding_dict

    def __len__(self):
        return len(self.data)

    def __getitem__(self,idx):
        row = self.data.iloc[idx]
        user = self.user_to_row[row['User']]
        book = self.item_to_row[row['Book']]
        rating = row['Rate'].astype('float32')
        social_embedding = self.social_embedding_dict.get(row['User'], np.zeros(200))
        return user, book, rating,social_embedding
        
        
        

In [114]:
# embedding_dim为超参数，由用户定义
class MatrixFactorization(nn.Module):
    def __init__(self,num_users,num_books,embedding_dim,hidden_state_social):
        super(MatrixFactorization,self).__init__()
        #词嵌入技术，将user和item分别嵌入为向量
        self.user_embeddings=nn.Embedding(num_users,embedding_dim)
        self.book_embeddings=nn.Embedding(num_books,embedding_dim)
        self.linear_embedding = nn.Linear(hidden_state_social,embedding_dim)
        self.output = nn.Linear(embedding_dim,6)

    def forward(self, user,book,social_embedding):
        # 输出即为user矩阵和book矩阵相乘得到的结果
        user_embedding=self.user_embeddings(user)
        book_embedding=self.book_embeddings(book)
        social_embedding_proj = self.linear_embedding(social_embedding)
        user_intergrate = user_embedding + social_embedding_proj
        return (user_intergrate * book_embedding).sum(dim = 1)



In [115]:
#TODO 创建训练集和测试集的数据集对象和数据加载器
train_data, test_data = train_test_split(loaded_data, test_size=0.2, random_state=42)

train_dataset = RatingDataset(train_data, user_to_row, item_to_row,social_embedding_dict)
test_dataset = RatingDataset(test_data, user_to_row, item_to_row,social_embedding_dict)

train_dataloader = DataLoader(train_dataset, batch_size=4096, shuffle=True, drop_last = True)
test_dataloader = DataLoader(test_dataset, batch_size=4096, shuffle=False, drop_last = True)

embedding_dim=32

In [116]:
num_users=len(user_ids)
num_books=len(item_ids)
print(num_users)
print(num_books)

4312
1200


In [117]:
hidden_state_social = 200
model=MatrixFactorization(num_users,num_books,embedding_dim,hidden_state_social).to(device)
criterion=nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

In [118]:
import numpy as np
model.train()
num_epochs = 20
lambda_b=0.001
lambda_u=0.001
for epoch in range(num_epochs):
    for user, book, rating,social_embedding in train_dataloader:
        optimizer.zero_grad()
        user = user.to(device)
        book = book.to(device)
        rating = rating.to(device)
        social_embedding = social_embedding.float().to(device)
        output = model(user, book,social_embedding)
        loss = criterion(output, rating) + lambda_u * model.user_embeddings.weight.norm(2) + lambda_b * model.book_embeddings.weight.norm(2)
        loss.backward()
        optimizer.step()
    # 监控损失或其他性能指标
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {loss.item()}")



Epoch 1/20, Loss: 24.907249450683594
Epoch 2/20, Loss: 10.737598419189453
Epoch 3/20, Loss: 6.514861106872559
Epoch 4/20, Loss: 4.485576152801514
Epoch 5/20, Loss: 3.1445844173431396
Epoch 6/20, Loss: 2.417938709259033
Epoch 7/20, Loss: 1.9984064102172852
Epoch 8/20, Loss: 1.6251544952392578
Epoch 9/20, Loss: 1.394412636756897
Epoch 10/20, Loss: 1.248272180557251
Epoch 11/20, Loss: 1.1179004907608032
Epoch 12/20, Loss: 1.037400245666504
Epoch 13/20, Loss: 0.9618353247642517
Epoch 14/20, Loss: 0.9101570844650269
Epoch 15/20, Loss: 0.8669217824935913
Epoch 16/20, Loss: 0.8442720174789429
Epoch 17/20, Loss: 0.8091962933540344
Epoch 18/20, Loss: 0.7860156893730164
Epoch 19/20, Loss: 0.786907434463501
Epoch 20/20, Loss: 0.7574633955955505


In [119]:

model.eval()  # 将模型设置为评估模式，不进行梯度更新
test_loss = 0.0
with torch.no_grad():  # 不计算梯度
    for user, book, rating,social_embedding in test_dataloader:
        user = user.to(device)
        book = book.to(device)
        rating = rating.to(device)
        social_embedding = social_embedding.float().to(device)
        output = model(user, book,social_embedding)
        loss = criterion(output, rating)
        #print(f"Test Loss: {loss}")
        test_loss += loss.item()

# 计算测试性能指标，例如均方误差或其他指标
average_test_loss = test_loss / len(test_dataloader)
print(f"Average Test Loss: {average_test_loss}")

Average Test Loss: 0.9313792021651017


In [120]:
from sklearn.metrics import ndcg_score
#使用余弦相似度计算预测顺序和实际数据的差距
model.eval()  # 将模型设置为评估模式，不进行梯度更新
num_users=int(num_users)
num_books=int(num_books)
real_score_array=np.zeros((num_users,num_books))
pred_score_array=np.zeros((num_users,num_books))

with torch.no_grad():  # 不计算梯度
    for user, book, rating,social_embedding in test_dataloader:
        user = user.to(device)
        book = book.to(device)
        social_embedding = social_embedding.float().to(device)
        rating = rating.to(device)
        pred = model(user, book,social_embedding)
        for i in range(4096):
            real_score_array[int(user[i]),int(book[i])]=int(rating[i])
            pred_score_array[int(user[i]),int(book[i])]=float(pred[i])
cos_sum=0
ndcg_sum=0
#抽取前100组user
for i in range(100):
    #cal ndcg
    ndcg=ndcg_score(real_score_array[i].reshape(1,-1),pred_score_array[i].reshape(1,-1))
    ndcg_sum=ndcg_sum+ndcg
    vec1=np.argsort(real_score_array[i])
    vec2=np.argsort(pred_score_array[i])
    cos_sim = vec1.dot(vec2) / (np.linalg.norm(vec1) * np.linalg.norm(vec2))
    cos_sum=cos_sum+cos_sim
print(f"cos similarity={cos_sum/100}\nNDCG_score={ndcg_sum/100}")


        

cos similarity=0.9970251450510551
NDCG_score=0.8957475528200479


In [121]:
from sklearn.metrics import ndcg_score
#print(real_score_array[0])
ndcg=ndcg_score(real_score_array[0].reshape(1,-1),pred_score_array[0].reshape(1,-1))
print(ndcg)

0.0
